## Load Libraries and import modules 

In [1]:
# Load all the vanila libraries 
import numpy as np
import pandas as pd
import os
import gc
from functools import reduce


# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Import tqdm for progress bar
from tqdm.auto import tqdm

# for timing functions
from timeit import default_timer as timer 

### Configure Project Parameters

In [2]:
# check your current directory
savepath = os.getcwd() + "\\data\\"

**Important:** Run the configuration file first `configs.py`. Importing this script and setting the seed and device parameters before importing any of the other modules ensures that evereything is sync.

**Important** If you want *change the configuration parameters*, change them before importing and running the pipeline. 

Now that all the configurations values are assigned globally, we can import the modules. If this is working, we expect each module to access the **same** **seed** and **device** we set. We are also expecting generated numbers **inside the modules** to be reproducible.

In [3]:
from models_util import ml_helper as mlh 

# Load Data 

## Dataframe with Features

Since there are differences between R and python, I tried to combine the advantages of both. In R due to vectorization, it is faster to compute euclidean, manhattan and cosinde distances of matrices (with missing Values too).<br> 
In python correlation analysis is way faster than R. So...<br>
- To speed up python i tried to create custom functions where the distances are calculated via tensor algebra (it speeds things up)
- I will be using another matrix from a previous analysis in R to validate that the computations are correct. <br>

So for all the feature engineering practices where we have Protein A - Protein B pairs, I used the following naming sceme:<br>

```
features_dataframe:
Var1 - Var2 - [proteomics_type]_[feature_type]_[feature_analysis] - db

```
**Explanations**:<br>
The **Var1** and **Var2** are the protein or sims pairs (unique pairs), where Var1 > Var2 (as a way to remove duplicates and self combinations) <br>
The **proteomics type** could be **SCBC** or **ABMS** (subcellular or total cell MS proteomics)<br>
The **feature_type** could be the **raw** MS signals, or the **VAE** embeddings, or the **umap** coordinates, <br> 
The **feature_analysis** could be correlations, distances, angles, or any other feature. If there is time I will perform a **feature importance analysis** in more detail. <br>
The **db** are the classes of the classification based on ground truth databases of protein-protein interractions, 0 or 1. If we create the features with the purpose of training a classifier. 

**Example** of a columns of some features: 'SCBC_raw_pearson', 'ABMS_vae_umap', 'ABMS_vae_euclidean' <br>  
**The features dataframe** containing these columns could have protein pairs, proteoform dominant pairs (sims), or all the possible proteoform combinations outhere.<br>

**NOTE**<br>
For 34 million unique pairs, 14 features require around 32 minutes and do not crash the RAM, that csv file is around 20gb size  

In [4]:
# an old file written in R for validation of engineering the features correctly 
# feat = pd.read_csv(os.getcwd() + "\\data\\processed\\merged_features_v1.txt", delimiter="\t")

In [5]:
# feat["db"] = np.where(feat["db"]=="F",0,1)

## Data from Embeddings (VAE, umap) and Raw signals for gene or peptide centric Feature Eng.

Here I will construct some features for **protein-protein interractions** or for the **dominant (sim) proteoforms**, which have a corresponding **gene symbol**.<br>
- It is the same approach for different protein tables.
- The difference lies whether I will use the groudtruth data during feature engineering.
- For a subset of protein-protein pairs, which will be used to train the classifier, i will use the ground truth pairs.
- For the whole protein matrices, the purpose is to just engineer the features for all possible protein/sims pairs.
- This will have impact to the memory of the computer. <br>

For the **feature engineering** function we want Dataframes:<br>
- where **index** is either gene symbol or any other annotations used. Each row corresponds to a protein, proteoform, etc..
- where the **columns** are counts or MS signals (preferable normalized and harmonized) or VAE latent variables or UMAP coordinates. 
- index is important since we generate pairs and we need to name them or follow them somehow. 

In [6]:
#### SOME GLOBAL VARIABLES TO AID THE ANALYSIS ######
ANALYSIS_LEVEL = "proteoform" # "protein" OR "sims" for the the dominant proteoforms
USE_GROUND_TRUTH = False # if set to TRUE, it subsets the tables based on gene symbols that exist within the PPI databases for every function below 
seed = 456


# USE 3 IF GROUND TRUTH IS TRUE, OTHERWISE 10 ARE ENOUGH 
BLOCKS = 13 # calculate in a way that each block contains 1000 rows/proteins. So if you have 10000, use 10 blocks, 


In [7]:
if ANALYSIS_LEVEL == "protein":

    # for subcell
    raw_scbc = pd.read_csv(os.getcwd() + "\\data\\features_protein\\protein_quant_merged.txt", delimiter="\t", index_col=0)
    vae_scbc = pd.read_csv(os.getcwd() + "\\data\\features_protein\\proteinscbc_latent.csv", index_col=0)
    umap_scbc = pd.read_csv(os.getcwd() + "\\data\\features_protein\\protein_scbc_umap.csv", index_col=0)

    # for total cell 
    raw_abms = pd.read_csv(os.getcwd() + "\\data\\features_protein\\prot_abms_norm.txt", delimiter="\t")
    vae_abms = pd.read_csv(os.getcwd() + "\\data\\features_protein\\protein_abms_latent.csv", index_col=0)
    umap_abms = pd.read_csv(os.getcwd() + "\\data\\features_protein\\protein_abms_umap.csv", index_col=0)

    # for ground truth, the combined CORUM and Compleat datasets in gene symbol format 
    pairs_df = pd.read_csv(os.getcwd() + "\\data\\processed\\" + "merged_pairs.txt", delimiter="\t")

elif ANALYSIS_LEVEL == "sims":
    # for subcell 
    raw_scbc = pd.read_csv(os.getcwd() + "\\data\\features_sim\\scbc_quant_sims.csv", index_col=0)
    vae_scbc = pd.read_csv(os.getcwd() + "\\data\\features_sim\\simscbc_latent.csv", index_col=0)
    umap_scbc = pd.read_csv(os.getcwd() + "\\data\\features_sim\\sim_scbc_umap.csv", index_col=0)

    # for total cell 
    raw_abms = pd.read_csv(os.getcwd() + "\\data\\features_sim\\abms_quant_sims.csv", index_col=0)
    vae_abms = pd.read_csv(os.getcwd() + "\\data\\features_sim\\sim_abms_latent.csv", index_col=0)
    umap_abms = pd.read_csv(os.getcwd() + "\\data\\features_sim\\sim_abms_umap.csv", index_col=0)

    # for ground truth, the combined CORUM and Compleat datasets in gene symbol format 
    pairs_df = pd.read_csv(os.getcwd() + "\\data\\processed\\" + "merged_pairs.txt", delimiter="\t")

elif ANALYSIS_LEVEL == "proteoform":
    # overwrite global parameters for proetoform analysis
    BLOCK = 13 # many proteoforms in the dataframes 
    USE_GROUND_TRUTH = False # There is no ground truth for proteoform analysis 

    # for subcell 
    raw_scbc = pd.read_csv(os.getcwd() + "\\data\\features_proteoforms\\SCBC2_proteoform_table 2.txt", index_col=0, delimiter="\t")
    vae_scbc = pd.read_csv(os.getcwd() + "\\data\\features_proteoforms\\whole_proteoformscbc_latent.csv", index_col=0)
    umap_scbc = pd.read_csv(os.getcwd() + "\\data\\features_proteoforms\\whole_proteoform_scbc_umap.csv", index_col=0)

    # filter out proteoform raw data 
    raw_scbc = raw_scbc.loc[raw_scbc.isna().sum(axis=1) < 0.3*raw_scbc.shape[1]]

    # for total cell 
    raw_abms = pd.read_csv(os.getcwd() + "\\data\\features_proteoforms\\ABMS_proteoform_table.txt", delimiter="\t", index_col=0)
    vae_abms = pd.read_csv(os.getcwd() + "\\data\\features_proteoforms\\whole_proteoform_abms_latent.csv", index_col=0)
    umap_abms = pd.read_csv(os.getcwd() + "\\data\\features_proteoforms\\whole_proteoform_abms_umap.csv", index_col=0)

else:
    raise ValueError("Choose either protein tables or dominant proteoform tables (sims) for analysis")

# check for duplicates in the indices and drop them (in case something survived)
for df in [raw_scbc, vae_scbc, umap_scbc, raw_abms, vae_abms, umap_abms]:
    print(f"Duplicates in {type(df)} before checking: {df.index.duplicated().sum()}")
    del df
    gc.collect()

# i found some in abms
raw_abms = raw_abms.loc[~raw_abms.index.duplicated(keep='first')]
vae_abms = vae_abms.loc[~vae_abms.index.duplicated(keep='first')]
umap_abms = umap_abms.loc[~umap_abms.index.duplicated(keep='first')]

Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0


In [8]:
# check for duplicates in the indices after clean up 
for df in [raw_scbc, vae_scbc, umap_scbc, raw_abms, vae_abms, umap_abms]:
    print(f"Duplicates in {type(df)} before checking: {df.index.duplicated().sum()}")
    del df
    gc.collect()

Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0
Duplicates in <class 'pandas.core.frame.DataFrame'> before checking: 0


In [17]:
raw_abms.head()

,X__POOL_A431_new___F1_Set4_tmt10plex_127C,X__POOL_A431_new___G1_Set6_tmt10plex_127C,X__POOL_A431_new___E1_Set2_tmt10plex_127C,X__POOL_Colo800_E1_Set1_tmt10plex_130N,X__POOL_COLO_800___F1_Set3_tmt10plex_130N,X__POOL_COLO_800___G1_Set5_tmt10plex_130N,X__POOL_Daudi___F1_Set4_tmt10plex_129N,X__POOL_Daudi___G1_Set6_tmt10plex_129N,X__POOL_Daudi___E1_Set2_tmt10plex_129N,X__POOL_DEL_E1_Set1_tmt10plex_129N,...,X__POOL_RH_30___G1_Set5_tmt10plex_130C,X__POOL_SK_N_BE_2____F1_Set4_tmt10plex_129C,X__POOL_SK_N_BE_2____G1_Set6_tmt10plex_129C,X__POOL_SK_N_BE_2____E1_Set2_tmt10plex_129C,X__POOL_SW403_E1_Set1_tmt10plex_127C,X__POOL_SW403___F1_Set3_tmt10plex_127C,X__POOL_SW403___G1_Set5_tmt10plex_127C,X__POOL_U_251_MG_new___F1_Set4_tmt10plex_130N,X__POOL_U_251_MG_new___G1_Set6_tmt10plex_130N,X__POOL_U_251_MG_new___E1_Set2_tmt10plex_130N
P_sim_ENSG00000005059,-0.213993,-0.137529,0.000000,-0.123314,-0.138273,-0.313949,0.764126,0.753426,0.649553,0.037584,...,0.719321,-1.441227,-1.380794,-0.776688,-0.756434,-0.470883,-0.710280,0.600169,0.410289,0.312207
P_sim_ENSG00000106066,0.000000,-0.016762,0.000000,0.901107,0.650345,1.136035,-0.642985,-0.217619,-0.381782,-0.337591,...,-0.296052,1.207977,0.903743,1.060591,-0.246002,-0.066274,-0.042394,0.323387,0.341632,0.257122
P_sim_ENSG00000162852,-0.285519,-0.230599,-0.298860,0.447177,0.362489,0.604464,0.414426,0.370486,0.448106,0.146033,...,-0.414919,0.416719,0.415692,0.414645,-0.028658,-0.033368,-0.033750,0.000000,-0.096140,0.009902
P_sim_ENSG00000198431,0.160704,0.178244,0.168050,1.032224,0.788027,0.928571,-0.090196,-0.132319,-0.121676,0.000000,...,-0.301409,0.077406,0.035310,0.014530,-0.221211,-0.322800,-0.235054,0.585081,0.662271,0.608153
P_sim_ENSG00000166037,-0.357024,-0.297056,-0.344701,-0.016710,0.000000,0.010441,0.098166,0.000000,0.000000,0.000000,...,-0.305652,0.667617,0.489154,0.686375,0.095432,0.050303,0.065703,-0.175670,-0.220072,-0.143094


In [16]:
# # set seed for reproducibility
# np.random.seed(seed)

# # take random samples around 8000 of the same indices for all the tables
# rand_index = raw_scbc.sample(5000, random_state=seed).index
# rand_index2 = raw_abms.sample(2000, random_state=seed).index

# # # subset the protein tables to same indices 
# # raw_scbc = raw_scbc.loc[rand_index]
# # vae_scbc = vae_scbc.loc[rand_index]
# # umap_scbc = umap_scbc.loc[rand_index]

# raw_abms = raw_abms.loc[rand_index2]
# vae_abms = vae_abms.loc[rand_index2]
# umap_abms = umap_abms.loc[rand_index2]

# # check if the order of the indices is the same 
# assert np.array_equal(raw_scbc.index, vae_scbc.index), "Indices of raw and VAE tables do not match"

In [11]:
# the dimensions of the tables before run
raw_abms.shape, vae_abms.shape, umap_abms.shape

((2000, 54), (2000, 30), (2000, 3))

# Feature Engineering for SubCell (ABMS" features dataframe)

In [12]:
if USE_GROUND_TRUTH:
    ground = pairs_df
    merge_cols = ["Var1", "Var2", "db"]
else:
    ground = None
    merge_cols = ["Var1", "Var2"]

# calculate correlation coefficients for all the pairs in raw and VAE tables 
scbc_raw_corr = mlh.correlation_blockwise(raw_abms,
                                          ground_truth= ground,
                                          data_name="ABMS_raw")

scbc_raw_corr.drop(columns="ABMS_raw_cor_pears", inplace=True)
gc.collect()

scbc_vae_corr = mlh.correlation_blockwise(vae_abms,
                                          ground_truth= ground,
                                          data_name="ABMS_vae")
gc.collect()


scbc_vae_corr.drop(columns="ABMS_vae_cor_pears", inplace=True)

scbc_raw_man = mlh.compute_distances_blockwise(raw_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_raw",
                                        n_chunks = BLOCKS,
                                        return_dist = "manhattan")
gc.collect()

scbc_raw_std = mlh.compute_distances_blockwise(raw_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_raw",
                                        n_chunks = BLOCKS,
                                        return_dist = "std_difference")
gc.collect()

scbc_raw_cos = mlh.compute_distances_blockwise(raw_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_raw",
                                        n_chunks = BLOCKS,
                                        return_dist = "cosine")
gc.collect()

scbc_umap_cos = mlh.compute_distances_blockwise(umap_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_umap",
                                        n_chunks = BLOCKS,
                                        return_dist = "cosine")
gc.collect()

scbc_umap_euc = mlh.compute_distances_blockwise(umap_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_umap",
                                        n_chunks = BLOCKS,
                                        return_dist = "euclidean")
gc.collect()

scbc_vae_cos = mlh.compute_distances_blockwise(vae_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_vae",
                                        n_chunks = BLOCKS,
                                        return_dist = "cosine")
gc.collect()


scbc_vae_std = mlh.compute_distances_blockwise(vae_abms,
                                        ground_truth = ground,
                                        data_name = "ABMS_vae",
                                        n_chunks = BLOCKS,
                                        return_dist = "std_difference")
gc.collect()


# merge the features with all the common values - no missing Values 
scbc_features = reduce(lambda left, right: pd.merge(left, right, on=merge_cols, how = "inner", validate="one_to_one"), [scbc_raw_corr, scbc_vae_corr, scbc_raw_man, scbc_raw_std, scbc_raw_cos, scbc_umap_cos, scbc_umap_euc, scbc_vae_cos, scbc_vae_std])

# clean memory
del scbc_raw_corr, scbc_vae_corr, scbc_raw_man, scbc_raw_std, scbc_raw_cos, scbc_umap_cos, scbc_umap_euc, scbc_vae_cos, scbc_vae_std
gc.collect()

# sanity check 
print(f"The feature dataframe has {scbc_features.shape[0]} rows and the following columns:\n {list(scbc_features.columns)}")
print(f"Missing Values in total in the features dataframe: {scbc_features.isna().sum().sum()}\n")


# # for consistancy separate and order columns 
# excluded_cols = [col for col in scbc_features.columns if col in merge_cols]
# other_cols = sorted([col for col in scbc_features.columns if col not in merge_cols])
# final_col_order = excluded_cols + other_cols

# # Reorder the DataFrame
# scbc_features = scbc_features[final_col_order]
# del excluded_cols, other_cols, final_col_order 


Analysis is starting for ABMS_raw to get pearson and spearman coefficients
------------------------------------------------------------------------------------------------------
Perform pairwise correlation analysis for the whole matrix - no ground truth will be used.
The number of pairs generated is 1999000
Analysis is completed

Analysis is starting for ABMS_vae to get pearson and spearman coefficients
------------------------------------------------------------------------------------------------------
Perform pairwise correlation analysis for the whole matrix - no ground truth will be used.
The number of pairs generated is 1999000
Analysis is completed

Analysis is starting for ABMS_raw for feature manhattan
------------------------------------------------------------------------------------------------------
Perform pairwise calculation for the whole matrix - no ground truth will be used.
The size of each block is 153 rows, and the number of blocks is 13
For total number of rows 2

In [13]:
# sanity check for the table 
scbc_features.head()

,Var1,Var2,ABMS_raw_cor_spear,ABMS_vae_cor_spear,ABMS_raw_man,ABMS_raw_std_dif,ABMS_raw_cos,ABMS_umap_cos,ABMS_umap_euc,ABMS_vae_cos,ABMS_vae_std_dif
0,P_sim_ENSG00000106066,P_sim_ENSG00000005059,-0.424503,-0.547052,58.393363,1.418823,1.404158,0.003457,3.412923,1.483711,0.914456
1,P_sim_ENSG00000162852,P_sim_ENSG00000005059,-0.007359,-0.094994,30.111108,0.698465,1.045987,0.004559,2.858024,1.078110,0.516638
2,P_sim_ENSG00000198431,P_sim_ENSG00000005059,-0.076118,-0.092770,34.483028,0.784448,1.030584,0.002448,2.260903,1.052668,0.547605
3,P_sim_ENSG00000166037,P_sim_ENSG00000005059,-0.140351,-0.237375,32.531869,0.741876,1.175268,0.006458,3.205279,1.204867,0.539839
4,P_sim_ENSG00000164292,P_sim_ENSG00000005059,-0.136364,-0.146607,37.151435,0.882057,1.102861,0.002412,2.378305,1.117200,0.612671


In [14]:
scbc_features.isna().sum(), scbc_features.shape

(Var1                  0
 Var2                  0
 ABMS_raw_cor_spear    0
 ABMS_vae_cor_spear    0
 ABMS_raw_man          0
 ABMS_raw_std_dif      0
 ABMS_raw_cos          0
 ABMS_umap_cos         0
 ABMS_umap_euc         0
 ABMS_vae_cos          0
 ABMS_vae_std_dif      0
 dtype: int64,
 (1999000, 11))

In [15]:
# which features to keep
tokeep = ['Var1',
          'Var2',
'ABMS_raw_cor_spear',
 'ABMS_raw_man',
 'ABMS_raw_std_dif',
 'ABMS_raw_cos',
 'ABMS_vae_cos',
 'ABMS_vae_cor_spear',
 'ABMS_umap_euc',
 'ABMS_umap_cos',
 'ABMS_vae_std_dif']

# change the features dataframe by keeping the columns aboves
scbc_features.loc[:, tokeep].head()


,Var1,Var2,ABMS_raw_cor_spear,ABMS_raw_man,ABMS_raw_std_dif,ABMS_raw_cos,ABMS_vae_cos,ABMS_vae_cor_spear,ABMS_umap_euc,ABMS_umap_cos,ABMS_vae_std_dif
0,P_sim_ENSG00000106066,P_sim_ENSG00000005059,-0.424503,58.393363,1.418823,1.404158,1.483711,-0.547052,3.412923,0.003457,0.914456
1,P_sim_ENSG00000162852,P_sim_ENSG00000005059,-0.007359,30.111108,0.698465,1.045987,1.078110,-0.094994,2.858024,0.004559,0.516638
2,P_sim_ENSG00000198431,P_sim_ENSG00000005059,-0.076118,34.483028,0.784448,1.030584,1.052668,-0.092770,2.260903,0.002448,0.547605
3,P_sim_ENSG00000166037,P_sim_ENSG00000005059,-0.140351,32.531869,0.741876,1.175268,1.204867,-0.237375,3.205279,0.006458,0.539839
4,P_sim_ENSG00000164292,P_sim_ENSG00000005059,-0.136364,37.151435,0.882057,1.102861,1.117200,-0.146607,2.378305,0.002412,0.612671


In [ ]:
if ANALYSIS_LEVEL == "protein":

    if USE_GROUND_TRUTH:
        scbc_features.to_csv(savepath + "features_protein\\abms_protein_features_db2.csv", header=True)
    else:
        scbc_features.to_csv(savepath + "features_protein\\abms_protein_features_full.csv", header=True)

elif ANALYSIS_LEVEL == "sims":
    
    if USE_GROUND_TRUTH:
        scbc_features.to_csv(savepath + "features_sim\\abms_sim_features_db2.csv", header=True)
    else:
        scbc_features.to_csv(savepath + "features_sim\\abms_sim_features_full.csv", header=True)

elif ANALYSIS_LEVEL == "proteoform":
    scbc_features.to_csv(savepath + "features_proteoforms\\abms_proteoform_features_full.csv", header=True)
    
else:
    raise RuntimeError("An ANALYSIS_LEVEL has not been specified")